# Base code for Scikit_learn_API 

In [11]:

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import os
import pickle
import numpy as np

In [8]:


batch_size = 100
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20


# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)


Using real-time data augmentation.
Epoch 1/10
500/500 [==============================] - 74s - loss: 1.7692 - acc: 0.3562 - val_loss: 1.3457 - val_acc: 0.5237
Epoch 2/10
500/500 [==============================] - 73s - loss: 1.3646 - acc: 0.5142 - val_loss: 1.0416 - val_acc: 0.6314
Epoch 3/10
500/500 [==============================] - 76s - loss: 1.1769 - acc: 0.5854 - val_loss: 0.8990 - val_acc: 0.6883
Epoch 4/10
500/500 [==============================] - 75s - loss: 1.0655 - acc: 0.6267 - val_loss: 0.8919 - val_acc: 0.6852
Epoch 5/10
500/500 [==============================] - 82s - loss: 0.9828 - acc: 0.6573 - val_loss: 0.7972 - val_acc: 0.7188
Epoch 6/10
500/500 [==============================] - 81s - loss: 0.9316 - acc: 0.6773 - val_loss: 0.7594 - val_acc: 0.7370
Epoch 7/10
500/500 [==============================] - 82s - loss: 0.8921 - acc: 0.6925 - val_loss: 0.7213 - val_acc: 0.7522
Epoch 8/10
500/500 [==============================] - 81s - loss: 0.8677 - acc: 0.7030 - val_loss

In [ ]:

# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break


Model Accuracy = 0.75
Actual Label = cat vs. Predicted Label = dog
Actual Label = ship vs. Predicted Label = automobile
Actual Label = ship vs. Predicted Label = ship
Actual Label = airplane vs. Predicted Label = horse
Actual Label = frog vs. Predicted Label = automobile
Actual Label = frog vs. Predicted Label = cat
Actual Label = automobile vs. Predicted Label = airplane
Actual Label = frog vs. Predicted Label = dog
Actual Label = cat vs. Predicted Label = horse
Actual Label = automobile vs. Predicted Label = truck
Actual Label = airplane vs. Predicted Label = bird
Actual Label = truck vs. Predicted Label = airplane
Actual Label = dog vs. Predicted Label = dog
Actual Label = horse vs. Predicted Label = truck
Actual Label = truck vs. Predicted Label = ship
Actual Label = ship vs. Predicted Label = airplane
Actual Label = dog vs. Predicted Label = horse
Actual Label = horse vs. Predicted Label = truck
Actual Label = ship vs. Predicted Label = airplane
Actual Label = frog vs. Predicted L

# Implementing the SCIKIT_LEARN_API

In [11]:

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import os
import pickle
import numpy as np

In [12]:


batch_size = 100
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20


# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
y_train = y_train.reshape(-1,)
y_train.shape
# Convert class vectors to binary class matrices.
def mycreate_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)
    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

    return model
# initiate RMSprop optimizer


# Let's train the model using RMSprop

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [13]:

if not data_augmentation:
    print('Not using data augmentation.')
    model = KerasClassifier(build_fn=mycreate_model, epochs= epochs, batch_size=batch_size, verbose=1)

else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model = KerasClassifier(build_fn=mycreate_model, epochs=epochs, batch_size=batch_size, verbose=1)

Using real-time data augmentation.


In [14]:
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=1337)
results = cross_val_score(model, x_train, y_train, cv=kfold)

Epoch 1/10
25000/25000 [==============================] - 34s - loss: 1.8798 - acc: 0.3181    
Epoch 2/10
25000/25000 [==============================] - 34s - loss: 1.4879 - acc: 0.4663    - ETA: 2s - loss: 1.48
Epoch 3/10
25000/25000 [==============================] - 34s - loss: 1.3009 - acc: 0.5402    
Epoch 4/10
25000/25000 [==============================] - 34s - loss: 1.1580 - acc: 0.5937    
Epoch 5/10
25000/25000 [==============================] - 34s - loss: 1.0618 - acc: 0.6256    
Epoch 6/10
25000/25000 [==============================] - 34s - loss: 0.9764 - acc: 0.6582    
Epoch 7/10
25000/25000 [==============================] - 34s - loss: 0.9104 - acc: 0.6826    
Epoch 8/10
25000/25000 [==============================] - 34s - loss: 0.8447 - acc: 0.7020    
Epoch 9/10
25000/25000 [==============================] - 34s - loss: 0.7926 - acc: 0.7246    
Epoch 10/10
25000/25000 [==============================] - 34s - loss: 1.8708 - acc: 0.3192    
Epoch 2/10
25000/25000 [===

# Grid Serach and reading the data from json file 

In [33]:

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.layers import Convolution2D, MaxPooling2D
import os
import pickle
import numpy as np



num_classes = 10
data_augmentation = True
num_predictions = 20

nb_filters = 10

nb_classes = 10
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

# The data, shuffled and split between train and test sets:

#(x_train, y_train), (x_test, y_test) = cifar10.load_data()
(x, y), (x_test, y_test) = cifar10.load_data()

# Spliting the data into TEST , TRAIN , VALIDATE 
x_train,x_valid,  y_train, y_valid = train_test_split(x, y, test_size=0.99, random_state=42,shuffle=False)
len(x_train), len(x_test), len(x_valid),len(y_train),len(y_test), len(y_valid)

img_rows, img_cols, img_colour = 32,32,3
x_train = x_train.reshape(-1, img_rows, img_cols, 3)
x_test = x_test.reshape(-1, img_rows, img_cols, 3)

input_shape = (img_rows, img_cols, 3)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
y_train = y_train.reshape(-1,)
y_train.shape

x_train shape: (500, 32, 32, 3)
500 train samples
10000 test samples


(500,)

In [34]:


# Convert class vectors to binary class matrices.
def create_model():
    model = Sequential()

    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],border_mode='valid',input_shape=input_shape))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128))

    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
    model.summary()


    return model
# initiate RMSprop optimizer


# Let's train the model using RMSprop

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [35]:

# @hidden_cell
credentials_1 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_ab4b66a6_b6e8_41f0_ba0d_2c0449850f6e',
  'project_id':'d6771321c8de4898b47d05baec0e24fe',
  'region':'dallas',
  'user_id':'e73ee618ded3475694661ae2e8d85d46',
  'domain_id':'89c8becdbf1a4cd79b15871915658e35',
  'domain_name':'1449725',
  'username':'member_961364bd71db2ce73ffe7ee07dbc9f49001d3163',
  'password':"""DNBLA7(RVt6&=#Yz""",
  'container':'DefaultProjectsekarvehuskyneuedu',
  'tenantId':'undefined',
  'filename':'datafile.json'
}

from io import BytesIO  
import requests  
import json  
import pandas as pd

def get_data(credentials):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', credentials['filename']])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return json.loads(resp2.content)

my_json = get_data(credentials_1)


In [36]:
 my_json.get("epoch")

[1, 2, 3]

In [37]:
seed = 7
np.random.seed(seed)
# load pima indians dataset

model2 = KerasClassifier(build_fn=create_model, verbose=0)
# grid search epochs, batch size and optimizer

epochs =  my_json.get("epoch")
batches = my_json.get("batch_size")
param_grid = dict( epochs=epochs, batch_size=batches)
print(param_grid)
grid = GridSearchCV(estimator=model2, param_grid=param_grid)
grid_result = grid.fit( x_train, y_train)

{'epochs': [1, 2, 3], 'batch_size': [1, 2, 3]}


/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (3, 3), padding="valid", input_shape=(32, 32, 3...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 30, 30, 10)        280       
_________________________________________________________________
activation_25 (Activation)   (None, 30, 30, 10)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 10)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 15, 10)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 2250)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               288128    
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1290      
__________

In [38]:
# the best combination 
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.300000 using {'epochs': 3, 'batch_size': 2}
0.178000 (0.036345) with: {'epochs': 1, 'batch_size': 1}
0.226000 (0.043598) with: {'epochs': 2, 'batch_size': 1}
0.254000 (0.028024) with: {'epochs': 3, 'batch_size': 1}
0.136000 (0.058176) with: {'epochs': 1, 'batch_size': 2}
0.234000 (0.020959) with: {'epochs': 2, 'batch_size': 2}
0.300000 (0.043318) with: {'epochs': 3, 'batch_size': 2}
0.174000 (0.031100) with: {'epochs': 1, 'batch_size': 3}
0.246000 (0.014699) with: {'epochs': 2, 'batch_size': 3}
0.232000 (0.045704) with: {'epochs': 3, 'batch_size': 3}


# Part 3- Building the model for 12000 randomly select image

In [1]:


from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import os
import pickle
import numpy as np
import os
import pickle
import numpy as np

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'
keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

In [4]:
import numpy as np
x_train=  np.concatenate((x_train, x_test), axis=0)
y_train=  np.concatenate((y_train, y_test), axis=0)


In [5]:
seed = 3
np.random.seed(seed)
randomsample = np.random.randint(0,60000,size=12000)
x_train = x_train[randomsample]
y_train = y_train[randomsample]


In [6]:
uniqueworddict={}
v= []
for i in range(0,len(y_train)): 
    a = y_train[i]
    v.extend(a)

for i in range(0,len(v)): 
    a = v[i]
    if a not in uniqueworddict:
        uniqueworddict[a] = 1   # Adding to the dictionary if the words are not present 
    else:
        uniqueworddict[a] += 1 
udict={}
for k, v in uniqueworddict.items():
            for i in range(0,len(labels['label_names'])): 

                v_labels = labels['label_names'][i]
                if k == i:
                    if v_labels not in udict:
                        udict[v_labels] = v   # Adding to the dictionary if the words are not present 
 # incrementing the count if the are present in the  dictionary

In [7]:
udict
 # incrementing the count if the are present in the  dictionary

{'airplane': 1200,
 'automobile': 1150,
 'bird': 1175,
 'cat': 1176,
 'deer': 1312,
 'dog': 1155,
 'frog': 1220,
 'horse': 1223,
 'ship': 1196,
 'truck': 1193}

In [8]:


batch_size = 100
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20


# The data, shuffled and split between train and test sets:

y_train = y_train.reshape(-1,)
y_train.shape
# Convert class vectors to binary class matrices.
def create_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(53, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(120, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)
    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

    return model
# initiate RMSprop optimizer


# Let's train the model using RMSprop

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [9]:

if not data_augmentation:
    print('Not using data augmentation.')
    model = KerasClassifier(build_fn=create_model, epochs= epochs, batch_size=batch_size, verbose=1)

else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=1)

Using real-time data augmentation.


In [10]:
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=1337)
results = cross_val_score(model, x_train, y_train, cv=kfold)

Epoch 1/10
5998/5998 [==============================] - 13s - loss: 2.1902 - acc: 0.1909    
Epoch 2/10
5998/5998 [==============================] - 12s - loss: 1.9444 - acc: 0.3013    
Epoch 3/10
5998/5998 [==============================] - 12s - loss: 1.7793 - acc: 0.3593    
Epoch 4/10
5998/5998 [==============================] - 12s - loss: 1.6454 - acc: 0.4140    
Epoch 5/10
5998/5998 [==============================] - 12s - loss: 1.5367 - acc: 0.4481    
Epoch 6/10
5998/5998 [==============================] - 12s - loss: 1.4661 - acc: 0.4687    
Epoch 7/10
5998/5998 [==============================] - 13s - loss: 1.3828 - acc: 0.5092    
Epoch 8/10
5998/5998 [==============================] - 12s - loss: 1.3254 - acc: 0.5258    
Epoch 9/10
5998/5998 [==============================] - 12s - loss: 1.2488 - acc: 0.5580    
Epoch 10/10
6002/6002 [==============================] - 13s - loss: 2.2133 - acc: 0.1829    
Epoch 2/10
6002/6002 [==============================] - 12s - loss: 1